Load jupyter sql extension


In [2]:
%load_ext sql
%config SqlMagic.autocommit = False

Next load the database

In [3]:
%sql sqlite:///../data/raw/frantamarit.db

'Connected: @../data/raw/frantamarit.db'

In [4]:
%%sql tables <<

SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

 * sqlite:///../data/raw/frantamarit.db
Done.
Returning data to local variable tables


In [5]:
tables.DataFrame()

,name
0,rank_info
1,personal_info
2,business_info


In [ ]:
"""
%%sql master_table <<

SELECT 
*
FROM rank_info
JOIN personal_info ON rank_info.id = rank_info.id
JOIN business_info ON business_info.id = rank_info.id;
"""

 * sqlite:///../data/raw/frantamarit.db
Done.


In [ ]:
1. Creating engine to connect to the database

In [25]:
from sqlalchemy import create_engine
import pandas as pd

In [32]:
""""
def get_dataframe(x):
    eng = create_engine(x)
    tables = eng.table_names()
    for i in enumerate(tables):
        i = pd.read_sql_table(i, con=eng)
    return 
    
get_dataframe('sqlite:///../data/raw/frantamarit.db')
""""

,id,name,position,Unnamed: 0
0,5542,aNANT ASAVABHOKHIN & FAMILY,NaN,0
1,4183,ARASH FERDOWSi,NaN,1
2,4761,banthoon LAMSAM & FAmily,NaN,2
3,1449,bilL ADDERLEY & FAMILY,NaN,3
4,1624,BORIS ZINGAREVICH,NaN,4
...,...,...,...,...
2255,9216,zhao XIAOQIANg,2204.0,2255
2256,5887,ZHOU LIANGZHANG,2205.0,2256
2257,8536,ZHU XINGMING,2206.0,2257
2258,2599,zhuO Jun,2207.0,2258


2. Create engine and get data tables

In [16]:
#Review path
eng = create_engine('sqlite:///../data/raw/frantamarit.db')
table_names = eng.table_names()
print(table_names)

['business_info', 'personal_info', 'rank_info']


3. Creating data frames for each table

In [18]:
rank_info = pd.read_sql_table('rank_info', con=eng)
personal_info = pd.read_sql_table('personal_info', con=eng)
business_info = pd.read_sql_table('business_info', con=eng)

In [37]:
rank_info.drop('Unnamed: 0', axis=1, inplace=True)
personal_info.drop('Unnamed: 0', axis=1, inplace=True)
business_info.drop('Unnamed: 0', axis=1, inplace=True)

In [42]:
business_info.head()

,id,Source,worth,worthChange,realTimeWorth,realTimePosition
0,6528,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,NaN,1
1,5875,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,NaN,2
2,8760,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,-0.002 millions USD,NaN,3
3,4685,Fashion & Retail ==> LVMH,72.0 BUSD,0.0 millions USD,NaN,4
4,8584,Technology ==> Facebook,71.0 BUSD,0.0 millions USD,NaN,5
